In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

books = pd.read_csv('BX-Books.csv', sep = ";", on_bad_lines = 'skip', encoding = 'latin-1')
books.head()
books.shape
books.columns
books = books[['ISBN','Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-L']]
books.head()
books.shape

books.rename(columns = {
    "Book-Title" : "title",
    "Book-Author" : "author",
    "Year-Of-Publication" : "year",
    "Publisher" : "publisher",
    "Image-URL-L" : "img_url"}, inplace = True)
books.head()

users = pd.read_csv('BX-Users.csv', sep = ";", on_bad_lines = 'skip', encoding = 'latin-1')
users.head()
users.shape
ratings = pd.read_csv('BX-Book-Ratings.csv', sep = ";", on_bad_lines = 'skip', encoding = 'latin-1')
ratings.head()
ratings.shape
print(books.shape)
print(users.shape)
print(ratings.shape)
ratings.rename(columns = {
    "User-ID" : "user_id",
    "Book-Rating" : "rating",}, inplace = True)

ratings.head()
ratings['user_id'].value_counts()
ratings['user_id'].unique().shape
x = ratings['user_id'].value_counts()> 200
x[x].shape
y = x[x].index
y

ratings = ratings[ratings['user_id'].isin(y)]
ratings.head()
ratings.shape
books.head()
ratings_with_books = ratings.merge(books, on = "ISBN")
ratings_with_books.head()
ratings_with_books.shape
num_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()
num_rating.head()
num_rating.rename(columns = {"rating" : "num_of_rating"}, inplace = True)
num_rating.head()
final_ratings = ratings_with_books.merge(num_rating, on = 'title')
final_ratings.head()
final_ratings.shape
final_ratings = final_ratings[final_ratings['num_of_rating'] >= 50]
final_ratings.head()
final_ratings.sample(10)
final_ratings.shape
final_ratings.drop_duplicates(['user_id', 'title'], inplace = True)
final_ratings.shape
book_pivot = final_ratings.pivot_table(columns = 'user_id', index = 'title', values = 'rating')
book_pivot
book_pivot.shape
book_pivot.fillna(0, inplace = True)
book_pivot
from scipy.sparse import csr_matrix
book_sparse = csr_matrix(book_pivot)
book_sparse
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm = 'brute')
model.fit(book_sparse)
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1), n_neighbors = 6)
distance
suggestion
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])
book_pivot.index[237]
books_name = book_pivot.index

import pickle
pickle.dump(model, open('artifacts/model.pkl','wb'))
pickle.dump(books_name , open('artifacts/books_name.pkl','wb'))
pickle.dump(final_ratings, open('artifacts/final_ratings.pkl','wb'))
pickle.dump(book_pivot, open('artifacts/book_pivot.pkl','wb'))

def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1), n_neighbors = 6)
    for i in range(len(suggestion)):
        books = book_pivot.index[suggestion[i]]
        for j in books:
            print(j)
book_name = 'Harry Potter and the Chamber of Secrets (Book 2)'
recommend_book(book_name)

/var/folders/7f/l_py_ngs7ds2yxsc8_1ddqm00000gn/T/ipykernel_24150/485481928.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('BX-Books.csv', sep = ";", on_bad_lines = 'skip', encoding = 'latin-1')


(271360, 6)
(278858, 3)
(1149780, 3)
Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')
Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Sorcerer's Stone (Book 1)
Exclusive
The Cradle Will Fall
